# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

In [4]:
# import libraries
import os
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
import pickle
import argparse

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/kamal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kamal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kamal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kamal/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
import pickle

In [7]:
# load data from database
engine = create_engine('sqlite:///figure8sqlite.db')
df = pd.read_sql_table("disaster_message", engine)
X = df['message']
Y = df.iloc[:, 4:]

In [8]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    '''
    Clean and tokenize the text in input
    Arg: text that needs to be tokenized
    Returns: tokenized text
    
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [10]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [11]:
tokenize("Weather update - a cold front from Cuba that c")

['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'c']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
##steps - 
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7fe3234ed8c0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=No

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [15]:

%%time
##Train pipeline
pipeline.fit(X_train, Y_train)

CPU times: user 6min 5s, sys: 2.34 s, total: 6min 8s
Wall time: 6min 14s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
Y_pred = pipeline.predict(X_test)


In [17]:
category_names = list(df.columns[4:])

In [18]:
for i in range(len(category_names)):
    print('Category: {} '.format(category_names[i]))
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))

Category: related 
              precision    recall  f1-score   support

           0       0.72      0.26      0.38      1558
           1       0.81      0.97      0.88      4996

    accuracy                           0.80      6554
   macro avg       0.76      0.61      0.63      6554
weighted avg       0.79      0.80      0.76      6554

Accuracy 0.7992065913945682


Category: request 
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5470
           1       0.90      0.41      0.56      1084

    accuracy                           0.89      6554
   macro avg       0.90      0.70      0.75      6554
weighted avg       0.89      0.89      0.88      6554

Accuracy 0.8941104668904486


Category: offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6517
           1       0.00      0.00      0.00        37

    accuracy                           0.99      6554
   macro avg 

/Users/kamal/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5695
           1       0.68      0.02      0.03       859

    accuracy                           0.87      6554
   macro avg       0.78      0.51      0.48      6554
weighted avg       0.85      0.87      0.81      6554

Accuracy 0.8700030515715593


Category: infrastructure_related 
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6136
           1       0.00      0.00      0.00       418

    accuracy                           0.94      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.88      0.94      0.91      6554

Accuracy 0.9354592615196826


Category: transport 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6242
           1       0.68      0.06      0.11       312

    accuracy                           0.95      6554
   macro avg 

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
%%time
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200], 
    'clf__estimator__min_samples_split': [2, 3, 4]

}
cv = GridSearchCV(pipeline, param_grid = parameters)


CPU times: user 113 µs, sys: 313 µs, total: 426 µs
Wall time: 452 µs


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
Y_train.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
21122,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,1
4376,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20904,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
%%time
cv.fit(X_train, Y_train)
#print('Best Parameters:', cv.best_params_)

CPU times: user 8h 37min 36s, sys: 6min 41s, total: 8h 44min 18s
Wall time: 1d 12h 49min 21s


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [25]:

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize))
            , ('tfidf', TfidfTransformer())]))
        , ('starting_verb', StartingVerbExtractor())]))
    , ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

In [104]:
%%time
pipeline.fit(X_train, Y_train)

CPU times: user 3min 54s, sys: 567 ms, total: 3min 55s
Wall time: 3min 55s


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [108]:
Y_pred = pipeline.predict(X_test)

print(classification_report(Y_test.iloc[:, 1:].values
                            , np.array([x[1:] for x in Y_pred])
                            , target_names = category_names))

                        precision    recall  f1-score   support

               related       0.79      0.55      0.65      1153
               request       0.00      0.00      0.00        21
                 offer       0.77      0.58      0.66      2731
           aid_related       0.61      0.27      0.38       514
          medical_help       0.62      0.29      0.40       334
      medical_products       0.67      0.21      0.32       177
     search_and_rescue       0.29      0.03      0.06       122
              security       0.71      0.34      0.46       212
              military       0.00      0.00      0.00         0
           child_alone       0.73      0.62      0.67       438
                 water       0.81      0.70      0.75       767
                  food       0.75      0.56      0.64       599
               shelter       0.62      0.38      0.47       105
              clothing       0.52      0.31      0.39       146
                 money       0.69      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [27]:
with open('model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.